# Fermentation data import and export

## 1. Imports and directory definitions

In [1]:
import pandas as pd # pandas for tabular data manipulation
import numpy as np  # numpy for numerical operations
import glob
import os
from pandas import ExcelWriter
import openpyxl

data_folder = '../../../data/'
fieldlist_path = '../fieldlist/fieldlist_v1.xlsx'
od_cal_file = '../2021-10-04_OD_calibration_file.xlsx'

## 2. Experient name definition

In [2]:
experiment = 'DDB_PD_066_AMBR'

## 3. Import data

### 3.1 Main cultures metadata

In [3]:
# get the path to experiment in the LIMS file folder
experiment_file = data_folder+'LIMS_data/'+experiment+'_LIMS.xlsx'
# import the data from the main culture sheet and drop any column that has only NaNs
meta_data = pd.read_excel(experiment_file, sheet_name='main culture').dropna(how='all', axis=1)
    
# cultures.index = range(0,len(cultures.index)) # updating the index of the dataframe
meta_data.rename(columns={'Sample ID': 'Parent Culture'}, inplace=True)
meta_data.sort_values('Reactor/Plate/Flask Number',inplace=True)
meta_data = meta_data.reset_index().drop('index',axis=1)
meta_data

,Parent Culture,Base Medium,Container ID (calculated),Container Type,Control?,Culture Type,Experiment,Experiment Annotation,growth_rate,Feed Medium #1,...,pH Control Base Solution,DO Control Setpoint (%),DO Control Cascade Level 1,DO Control Cascade Level 2,DO Control Cascade Level 3,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type
0,DDB_PD_066_AMBR_DDB7-006_R13_Main_,DDB_BM_005,R13,AMBR 250,Yes,Main,DDB_PD_066_AMBR,DDB TF KO,0.11,DDB_FM_005,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
1,DDB_PD_066_AMBR_SDT177-001_R15_Main_,DDB_BM_005,R15,AMBR 250,No,Main,DDB_PD_066_AMBR,DDB TF KO,0.11,DDB_FM_005,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
2,DDB_PD_066_AMBR_SDT177-001_R16_Main_,DDB_BM_005,R16,AMBR 250,No,Main,DDB_PD_066_AMBR,DDB TF KO,0.11,DDB_FM_005,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
3,DDB_PD_066_AMBR_SDT178-001_R17_Main_,DDB_BM_005,R17,AMBR 250,No,Main,DDB_PD_066_AMBR,DDB TF KO,0.11,DDB_FM_005,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
4,DDB_PD_066_AMBR_SDT178-001_R18_Main_,DDB_BM_005,R18,AMBR 250,No,Main,DDB_PD_066_AMBR,DDB TF KO,0.11,DDB_FM_005,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
5,DDB_PD_066_AMBR_SDT179-001_R19_Main_,DDB_BM_005,R19,AMBR 250,No,Main,DDB_PD_066_AMBR,DDB TF KO,0.11,DDB_FM_005,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
6,DDB_PD_066_AMBR_SDT179-001_R20_Main_,DDB_BM_005,R20,AMBR 250,No,Main,DDB_PD_066_AMBR,DDB TF KO,0.11,DDB_FM_005,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
7,DDB_PD_066_AMBR_SDT180-001_R21_Main_,DDB_BM_005,R21,AMBR 250,No,Main,DDB_PD_066_AMBR,DDB TF KO,0.11,DDB_FM_005,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
8,DDB_PD_066_AMBR_SDT180-001_R22_Main_,DDB_BM_005,R22,AMBR 250,No,Main,DDB_PD_066_AMBR,DDB TF KO,0.11,DDB_FM_005,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
9,DDB_PD_066_AMBR_SDT189-001_R23_Main_,DDB_BM_005,R23,AMBR 250,No,Main,DDB_PD_066_AMBR,DDB TF KO,0.11,DDB_FM_005,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air


### 3.1 Online data

In [4]:
online_data_dir = 'online_data_interpolated/'
csv_prefix = '_online_stacked_2m.csv'

In [5]:
# import fieldlist
fieldlist = pd.read_excel(fieldlist_path)
# initialize an empty DataFrame with columns from the variables specified in the fieldlist
online_data = pd.DataFrame(columns=list(fieldlist.new_variable_name))

In [6]:
ambr_df = pd.read_csv(data_folder+ online_data_dir + str(experiment) + csv_prefix)

for column in list(online_data.columns): # iterate over the columns of the temporary dataframe. The columns are the variable names of the online data.
    # match the column name (new_variable_name) to the name given by ambr250. Then assign the ambr250 online data to the matching column in the temp_df
    online_data[column] = ambr_df[fieldlist[fieldlist.new_variable_name == column].AMBR_name.iloc[0]]

# adding a column with the experiment name
online_data['Experiment'] = experiment

# Converting the bioreactor naming format in the temporary ambr df
for i in range(1,10):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R0'+str(i))
for i in range(10,25):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R'+str(i))

# adding a column with the Reactor Number
online_data['Reactor/Plate/Flask Number'] = ambr_df['Batch ID'] # adding a column with the reactor ID
    
# dropping some unnecessary rows from the online data DataFrame:
online_data.dropna(axis=0, subset=['Inlet_air_O2'],inplace=True)
    
# updating index
online_data.index = range(0,len(online_data.index)) # updating the index of the dataframe
    
# Merging online data with cultures metadata
online_data = pd.merge(left=online_data,right=meta_data)
    
# phase annotation
feed_vol_list = online_data['Feed_volume'].tolist()
phase_list = ['feed' if i > 0.0 else 'batch' for i in feed_vol_list]
online_data['Phase'] = phase_list
online_data.head()

,Time,DO,pH,Stirrer_speed,Temperature,Pressure,Air_flow,Oxygen_flow,Inlet_air_O2,Offgas_O2,...,DO Control Setpoint (%),DO Control Cascade Level 1,DO Control Cascade Level 2,DO Control Cascade Level 3,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type,Phase
0,0.000000,98.220401,7.137991,998.005941,30.010000,3.331985,99.981895,0.0,20.95,20.667000,...,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air,batch
1,0.033333,98.309201,7.137632,1000.263443,30.010000,3.150215,99.888668,0.0,20.95,20.665895,...,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air,batch
2,0.066667,98.337342,7.137665,998.993902,30.010000,3.209695,100.124333,0.0,20.95,20.667997,...,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air,batch
3,0.100000,98.342056,7.140282,997.974994,30.010000,3.111907,99.888740,0.0,20.95,20.669436,...,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air,batch
4,0.133333,98.514331,7.138678,1000.869608,30.012261,3.138670,99.998215,0.0,20.95,20.667000,...,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air,batch


### 3.2 Offline data

#### 3.2.1 Timepoint data

In [7]:
timepoint_cols = ['Entity','Experiment','Parent Culture','Reactor/Plate Number','Timepoint (#)','Timepoint (h)','Volume','OD']
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experimentin the LIMS file folder

timepoint_data = pd.read_excel(file, sheet_name='timepoint samples')[timepoint_cols] # importing timepoint data from one experiment into a temporary DFF

# renaming some of the columns in the dataframe and reindexing
timepoint_data.rename(columns={"Entity": "Timepoint Sample", "Reactor/Plate Number": "Reactor/Plate/Flask Number", "Volume": "Sample volume (mL)"},inplace=True)
timepoint_data = timepoint_data.reset_index().drop('index',axis=1)
timepoint_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD
0,DDB_PD_066_AMBR_R13__S00,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,R13,S00,0.000000,1.5,0.332
1,DDB_PD_066_AMBR_R13__S01,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,R13,S01,3.019057,1.5,0.372
2,DDB_PD_066_AMBR_R13__S02,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,R13,S02,6.018412,1.5,0.848
3,DDB_PD_066_AMBR_R13__S03,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,R13,S03,9.017413,3.5,2.610
4,DDB_PD_066_AMBR_R13__S04,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,R13,S04,12.018493,1.5,6.640


#### 3.2.2 Analytical data

In [8]:
analytical_cols = ['Name', 'Analysis Type', 'Cellular Compartment', 'Dilution Factor','Experiment', 'Parent Culture', 'Sample Prep Date', 'Timepoint (h)', 'Timepoint Sample', 'compound_name', 'concentration','concentration_unit']
    
analytical_data = pd.DataFrame(columns=analytical_cols) # initializing an empty DF for timepoint_data
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experiment in the LIMS file folder

analytical_data = pd.read_excel(file, sheet_name='analytical samples')[analytical_cols] # importing timepoint data from one experiment into a temporary DF

# renaming one of the columns in the dataframe and reindexing
analytical_data.rename(columns={"Name": "Analytical Sample"},inplace=True)
analytical_data = analytical_data.reset_index().drop('index',axis=1)

# correcting for the dilution factor, then dropping the dilution factor
analytical_data['concentration'] = analytical_data['concentration'] * analytical_data['Dilution Factor']
analytical_data.drop(['Dilution Factor'], axis=1, inplace=True)
# removing any analytical data not obtained from a main culture sample
analytical_data = analytical_data[analytical_data['Parent Culture'].notnull()]
analytical_data.head()

,Analytical Sample,Analysis Type,Cellular Compartment,Experiment,Parent Culture,Sample Prep Date,Timepoint (h),Timepoint Sample,compound_name,concentration,concentration_unit
0,DDB_PD_066_AMBR_R13__S09_SOA_#1,SOA,Extracellular Region,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,2021-09-20,29.46422,DDB_PD_066_AMBR_R13__S09,succinic acid,0.473483,g/L
1,DDB_PD_066_AMBR_R13__S09_SOA_#1,SOA,Extracellular Region,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,2021-09-20,29.46422,DDB_PD_066_AMBR_R13__S09,ethanol,0.000000,g/L
2,DDB_PD_066_AMBR_R13__S09_SOA_#1,SOA,Extracellular Region,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,2021-09-20,29.46422,DDB_PD_066_AMBR_R13__S09,citric acid,0.655737,g/L
3,DDB_PD_066_AMBR_R13__S09_SOA_#1,SOA,Extracellular Region,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,2021-09-20,29.46422,DDB_PD_066_AMBR_R13__S09,D-glucose,0.000000,g/L
4,DDB_PD_066_AMBR_R13__S09_SOA_#1,SOA,Extracellular Region,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,2021-09-20,29.46422,DDB_PD_066_AMBR_R13__S09,formic acid,0.000000,g/L


##### 3.2.2.1 Pivot analytical data

In [9]:
analytical_data['compound_name_and_unit'] = analytical_data['compound_name'] + ' [' + analytical_data['concentration_unit'] + ']'

analytical_data_pivot = analytical_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
analytical_data_pivot['Timepoint Sample'] = analytical_data_pivot.index

analytical_data_pivot.index.name = None
analytical_data_pivot.columns.name = None
analytical_data_pivot = analytical_data_pivot.reset_index().drop('index',axis=1)

analytical_data_pivot.head()

,D-glucose [g/L],acetic acid [g/L],citric acid [g/L],ethanol [g/L],formic acid [g/L],lactate [g/L],pyruvic acid [g/L],succinic acid [g/L],Timepoint Sample
0,13.256095,0.000000,0.943843,0.000000,0.000000,0.000000,0.000000,0.000000,DDB_PD_066_AMBR_R13__S00
1,12.893695,0.000000,0.927784,0.000000,0.000000,0.000000,0.000000,0.000000,DDB_PD_066_AMBR_R13__S01
2,12.729391,0.126317,0.951996,0.000000,0.000000,0.093745,0.000000,0.000000,DDB_PD_066_AMBR_R13__S02
3,11.237730,0.285690,0.971394,0.000000,0.310888,0.486602,0.203232,0.000000,DDB_PD_066_AMBR_R13__S03
4,8.584424,0.468459,0.929586,0.130837,0.509007,0.940181,0.318524,0.090074,DDB_PD_066_AMBR_R13__S04


#### 3.2.3 Trace_metal_data

In [10]:
trace_metal_data = pd.read_excel(data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx', sheet_name='trace metal export')
trace_metal_data.rename(columns={'Name': 'Analytical Sample'}, inplace=True)

# timepoint_samples_temp = []
# for i in df_tm['Analytical Sample'].to_list():
#     timepoint_samples_temp.append(i.split('_TM')[0])

# df_tm['Timepoint Sample'] = timepoint_samples_temp
trace_metal_data['Timepoint Sample'] = trace_metal_data['Analytical Sample'].apply(lambda x: pd.Series(str(x).split("_TM")[0]))
trace_metal_data.head()

,Analytical Sample,compound_name,concentration,concentration_unit,Timepoint Sample
0,DDB_PD_066_AMBR_R13__S00_TM_#1,co,498.223790,µg/L,DDB_PD_066_AMBR_R13__S00
1,DDB_PD_066_AMBR_R13__S00_TM_#1,cu,477.652275,µg/L,DDB_PD_066_AMBR_R13__S00
2,DDB_PD_066_AMBR_R13__S00_TM_#1,fe,41033.631270,µg/L,DDB_PD_066_AMBR_R13__S00
3,DDB_PD_066_AMBR_R13__S00_TM_#1,k,5242.221466,mg/L,DDB_PD_066_AMBR_R13__S00
4,DDB_PD_066_AMBR_R13__S00_TM_#1,mg,0.000000,mg/L,DDB_PD_066_AMBR_R13__S00


##### 3.2.2.1 Pivot trace metal data

In [11]:
trace_metal_data['compound_name_and_unit'] = trace_metal_data['compound_name'] + ' [' + trace_metal_data['concentration_unit'] + ']'

trace_metal_data_pivot = trace_metal_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
trace_metal_data_pivot['Timepoint Sample'] = trace_metal_data_pivot.index

trace_metal_data_pivot.index.name = None
trace_metal_data_pivot.columns.name = None
trace_metal_data_pivot = trace_metal_data_pivot.reset_index().drop('index',axis=1)

trace_metal_data_pivot.head()

,co [µg/L],cu [µg/L],fe [µg/L],k [mg/L],mg [mg/L],mn [µg/L],mo [µg/L],na [mg/L],s [mg/L],zn [µg/L],Timepoint Sample
0,498.223790,477.652275,41033.63127,5242.221466,0.000000,347.343236,816.067438,51.142502,2109.687151,5913.655084,DDB_PD_066_AMBR_R13__S00
1,497.979850,465.798505,39872.58813,5450.907286,30.608764,353.344390,797.098203,43.577786,2210.630142,5836.733666,DDB_PD_066_AMBR_R13__S01
2,493.693037,481.789113,41374.83405,5234.506307,0.000000,253.931288,829.227726,39.216857,2117.408148,5963.147000,DDB_PD_066_AMBR_R13__S02
3,483.028484,454.410551,40007.26887,5346.217852,0.000000,351.033643,791.660177,31.834029,2125.591999,5766.184877,DDB_PD_066_AMBR_R13__S03
4,499.704317,470.221198,41451.14912,5060.536387,24.642805,357.606793,804.748679,35.072473,2099.071792,5924.458414,DDB_PD_066_AMBR_R13__S04


#### 3.2.4 Custom data

In [12]:
try:
    custom_data = pd.read_excel(data_folder+'/custom_data_import_files/'+experiment+'_custom.xlsx')
    custom_data.head()
except:
    custom_data = pd.DataFrame(columns=['Timepoint Sample'])
    
custom_data

,Timepoint Sample


#### 3.2.5 Merging all offline data

In [13]:
ac_merge = pd.merge(left=analytical_data_pivot, right=trace_metal_data_pivot, how='left')
custom_merge = pd.merge(left=ac_merge, right=custom_data, how='left')
offline_data = pd.merge(left=timepoint_data, right=custom_merge, how='left')
offline_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD,D-glucose [g/L],acetic acid [g/L],...,co [µg/L],cu [µg/L],fe [µg/L],k [mg/L],mg [mg/L],mn [µg/L],mo [µg/L],na [mg/L],s [mg/L],zn [µg/L]
0,DDB_PD_066_AMBR_R13__S00,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,R13,S00,0.000000,1.5,0.332,13.256095,0.000000,...,498.223790,477.652275,41033.63127,5242.221466,0.000000,347.343236,816.067438,51.142502,2109.687151,5913.655084
1,DDB_PD_066_AMBR_R13__S01,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,R13,S01,3.019057,1.5,0.372,12.893695,0.000000,...,497.979850,465.798505,39872.58813,5450.907286,30.608764,353.344390,797.098203,43.577786,2210.630142,5836.733666
2,DDB_PD_066_AMBR_R13__S02,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,R13,S02,6.018412,1.5,0.848,12.729391,0.126317,...,493.693037,481.789113,41374.83405,5234.506307,0.000000,253.931288,829.227726,39.216857,2117.408148,5963.147000
3,DDB_PD_066_AMBR_R13__S03,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,R13,S03,9.017413,3.5,2.610,11.237730,0.285690,...,483.028484,454.410551,40007.26887,5346.217852,0.000000,351.033643,791.660177,31.834029,2125.591999,5766.184877
4,DDB_PD_066_AMBR_R13__S04,DDB_PD_066_AMBR,DDB_PD_066_AMBR_DDB7-006_R13_Main_,R13,S04,12.018493,1.5,6.640,8.584424,0.468459,...,499.704317,470.221198,41451.14912,5060.536387,24.642805,357.606793,804.748679,35.072473,2099.071792,5924.458414


## 4. Export data

### 4.2 Separate csv files

In [14]:
meta_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_meta_data_export.csv',index=False)
offline_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_offline_data_export.csv',index=False)
online_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_online_data_export.csv',index=False)